In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, roc_auc_score
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

##df = pd.read_csv("C:/Users/sonam/PycharmProjects/ExamPred/kaggle/input/students-exam-scores/Expanded_data_with_more_features.csv")
df = pd.read_csv("C:/Users/tabas/OneDrive-VanierCollege/Documents/GitHub/ExamPred/kaggle/input/students-exam-scores/Expanded_data_with_more_features.csv")
df.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           30641 non-null  int64  
 1   Gender               30641 non-null  object 
 2   EthnicGroup          28801 non-null  object 
 3   ParentEduc           28796 non-null  object 
 4   LunchType            30641 non-null  object 
 5   TestPrep             28811 non-null  object 
 6   ParentMaritalStatus  29451 non-null  object 
 7   PracticeSport        30010 non-null  object 
 8   IsFirstChild         29737 non-null  object 
 9   NrSiblings           29069 non-null  float64
 10  TransportMeans       27507 non-null  object 
 11  WklyStudyHours       29686 non-null  object 
 12  MathScore            30641 non-null  int64  
 13  ReadingScore         30641 non-null  int64  
 14  WritingScore         30641 non-null  int64  
dtypes: float64(1), int64(4), object(10)


In [2]:
#1. Check and Clean Data
# Convert 'NrSiblings' to integer
df['NrSiblings'] = df['NrSiblings'].astype(pd.Int64Dtype())

# Check unique values in each column
for column in df.columns:
    print(f"{column}: {df[column].unique()}")

# Check null values
print("\nNull Values in Columns:")
print(df.isnull().sum())

Unnamed: 0: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  30  31  32  33  34  35  36
  37  38  39  40  41  43  44  45  47  49  50  51  52  53  54  55  56  57
  58  59  61  62  64  65  66  67  68  69  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  87  88  89  91  92  93  94  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 148 149 150 152 153 154 155
 156 157 158 159 160 161 162 163 164 165 166 167 168 170 171 172 173 174
 175 176 177 178 179 180 181 182 183 185 186 187 188 189 190 191 192 193
 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211
 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229
 230 231 232 233 234 235 236 237 238 239 240 241 243 244 245 246 247 248
 249 250 251 252 253 254 255 256 257 26

In [3]:
#2. Impute Missing Values
# Fill missing values for categorical columns with mode
categorical_cols = ['EthnicGroup', 'WklyStudyHours', 'ParentEduc', 
                    'ParentMaritalStatus', 'IsFirstChild', 
                    'PracticeSport', 'TestPrep', 'TransportMeans']
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Fill missing numerical values
df['NrSiblings'] = df['NrSiblings'].interpolate()

# Confirm no null values remain
print("\nAfter Imputation:")
print(df.isnull().sum())



After Imputation:
Unnamed: 0             0
Gender                 0
EthnicGroup            0
ParentEduc             0
LunchType              0
TestPrep               0
ParentMaritalStatus    0
PracticeSport          0
IsFirstChild           0
NrSiblings             0
TransportMeans         0
WklyStudyHours         0
MathScore              0
ReadingScore           0
WritingScore           0
dtype: int64


In [4]:
#3. Map Binary and Ordinal Variables
# Map binary and ordinal features
binary_mappings = {
    'Gender': {'female': 0, 'male': 1},
    'TestPrep': {'none': 0, 'completed': 1},
    'IsFirstChild': {'no': 0, 'yes': 1},
    'TransportMeans': {'private': 0, 'school_bus': 1}
}

df.replace(binary_mappings, inplace=True)

# Map ordinal features (if any, e.g., study hours)
study_hours_mapping = {'< 5': 'Less than 5 h', '5 - 10': 'Between 5-10 h', '> 10': 'More than 10 h'}
df['WklyStudyHours'] = df['WklyStudyHours'].map(study_hours_mapping)


In [8]:
#4. Encode Categorical Variables
# One-hot encode categorical variables
categorical_features = ['EthnicGroup', 'LunchType', 'ParentMaritalStatus', 'WklyStudyHours', 'ParentEduc']
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Confirm final dataframe structure
print("\nFinal Dataset Columns:")
print(df.columns)


KeyError: "['EthnicGroup', 'LunchType', 'ParentMaritalStatus', 'WklyStudyHours'] not in index"

In [ ]:
#Visualize Data Distributions
import seaborn as sns
import matplotlib.pyplot as plt

for column in df.columns:
    plt.figure(figsize=(5, 3))
    if pd.api.types.is_numeric_dtype(df[column]):
        sns.histplot(df[column], kde=False, bins=30, color='blue')
        plt.title(f'Distribution of {column}', fontsize=16)
    else:
        df[column].value_counts().plot(kind='bar', color='green')
        plt.title(f'Value Counts of {column}', fontsize=12)
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()


In [ ]:
#6. Scaling and Splitting Data 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target
X = df.drop('MathScore', axis=1)
y = df['MathScore']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data Preprocessing Complete.")
